In [ ]:
#pip install --upgrade google-api-python-client
#pip install --upgrade google-cloud-storage
#pip install --upgrade google-cloud-spanner

In [1]:
import pandas as pd
import ibm_db_dbi as idb
from sqlalchemy import create_engine
import pyodbc
import warnings
import pyvalidata as pvd
import hashlib
#import time 
#import datetime
from datetime import datetime
import numpy as np
import sqlalchemy
import ibm_db_sa
import csv
from google.cloud import storage
from google.cloud import spanner
import os
import pandas as pd
from google.cloud import spanner

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="kr-7738-supvis-t-test-app-deployer-sa.json"

In [3]:
#projects/kr-7738-supvis-t/instances/nc-event-sink-instance/databases/storewalks-db

# Your Cloud Spanner instance ID.
instance_id = "nc-event-sink-instance"

# Your Cloud Spanner database ID.
database_id_1 = "udm-db"
database_id_2 = "storewalks-db"

# Instantiate a client.
spanner_client = spanner.Client()

# Get a Cloud Spanner instance by ID.
instance = spanner_client.instance(instance_id)

# Get a Cloud Spanner database by ID (for database_id_1).
database_1 = instance.database(database_id_1)
print("Connection to udm-db (database 1) successful.")

# Get a Cloud Spanner database by ID (for database_id_1).
database_2 = instance.database(database_id_2)
print("Connection to storewalks-db (database 2) successful.")

Connection to udm-db (database 1) successful.
Connection to storewalks-db (database 2) successful.


## 1) Daily Data Validation between source and target ##

In [ ]:
#database_id_1 = "udm-db"  & instance_id = "nc-event-sink-instance" (Source Query)

#Source Data
with database_1.snapshot() as snapshot:
    result = snapshot.execute_sql("""
        SELECT * FROM (
            SELECT
                div_str_id AS div_str_id_s,
                task_type_name,
                task_classification AS task_classification_s,
                a.task_desc,
            FROM
                (SELECT * FROM santn_rotn_task_mstr) a
                JOIN (SELECT * FROM sub_dpmt_mpng) b USING (sub_dpmt_id)
                JOIN (SELECT * FROM str_dpmt_mpng) c ON c.dpmt_name = b.prmry_dpmt_name
                LEFT JOIN (SELECT * FROM rolng_task_addtnl_info) d
                    ON (a.santn_rotn_task_mstr_id = d.santn_rotn_task_mstr_id)
            WHERE
                a.is_actv = 'Y'
                AND c.is_actv = 'Y'
                and DATE(strt_date) >='2023-09-05' 
                and DATE(strt_date) <='2023-09-05' 
                AND task_classification LIKE 'Daily'
                AND div_str_id = '53100515'
            UNION ALL
            SELECT
                div_str_id AS div_str_id_s,
                task_type_name,
                task_classification AS task_classification_s,
                a.task_desc,
            FROM
                (SELECT * FROM santn_rotn_task_mstr) a
                JOIN (SELECT * FROM sub_dpmt_mpng) b USING (sub_dpmt_id)
                JOIN (SELECT * FROM str_dpmt_mpng) c ON c.dpmt_name = b.prmry_dpmt_name
                LEFT JOIN (SELECT * FROM santn_rotn_task_schdl) d
                    ON (a.santn_rotn_task_mstr_id = d.santn_rotn_task_mstr_id)
            WHERE
                a.is_actv = 'Y'
                AND c.is_actv = 'Y'
                and DATE(schd_strt_date) >='2023-09-05' 
                and DATE(schd_strt_date) <='2023-09-05'
                AND task_classification LIKE 'Daily'
                AND div_str_id = '53100515'
        ) Daily_S
    """)
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_s_df = pd.DataFrame(rows, columns = cols)

In [ ]:
num_rows, num_columns = result_s_df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")
result_s_df.head()

In [ ]:
#database_id_2 = "storewalks-db"  & instance_id = "nc-event-sink-instance"
#Target Data
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
        SELECT
            CONCAT(division_number, store_number) AS div_str_id,
            task_type,
            task_classification,
            task_description,
        FROM
            task
        WHERE
            task_type <> 'Green Rack'
            AND task_classification LIKE 'Daily'
            AND task_date = '2023-09-05'
            AND CONCAT(division_number, store_number) = '53100515'
    """)
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_t_df = pd.DataFrame(rows, columns=cols)


In [ ]:
num_rows, num_columns = result_t_df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")
result_t_df.head()

In [ ]:
# Merge the two DataFrames on 'div_str_id' and 'task_desc'
merged_df = pd.merge(result_s_df, result_t_df, left_on=['div_str_id_s', 'task_desc'], right_on=['div_str_id', 'task_description'], how='outer')

# Create a new column 'ComparisonResult' based on your conditions
merged_df['ComparisonResult'] = 'Mismatch'

# Condition 1: Matched
condition_matched = (
    (merged_df['task_type_name'] == merged_df['task_type']) &
    (merged_df['task_classification_s'] == merged_df['task_classification']) &
    #(merged_df['sub_dpmt_name'] == merged_df['department_name']) &
    (merged_df['task_desc'] == merged_df['task_description'])
    #(merged_df['concatenated_s'] == merged_df['concatenated_t'])
)
merged_df.loc[condition_matched, 'ComparisonResult'] = 'Matched'

# Condition 2: Record NOT in Target
condition_not_in_target = (merged_df['div_str_id_s'].notna()) & (merged_df['div_str_id'].isna())
merged_df.loc[condition_not_in_target, 'ComparisonResult'] = 'Record NOT in Target'

# Condition 3: Record NOT in Source
condition_not_in_source = (merged_df['div_str_id'].notna()) & (merged_df['div_str_id_s'].isna())
merged_df.loc[condition_not_in_source, 'ComparisonResult'] = 'Record NOT in Source'

# Remove duplicates from merged_df
merged_df = merged_df.drop_duplicates()

# Reset the index after removing duplicates (optional)
merged_df.reset_index(drop=True, inplace=True)

# Filter records where 'ComparisonResult' is not 'Matched'
failed_records = merged_df[merged_df['ComparisonResult'] != 'Matched']

# Check if there are any failed records
if not failed_records.empty:
    print('Daily Validation Report Failed')

    # Get the current date as a timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    # Define the full path to save the report file
    file_path = r'C:\\Users\\AS81970\\Desktop\\2023 Q3\\Store Walk-in\\Reports'
    
    # Define the file name with the date appended
    report_file_name = f'Daily_Validation_Report_Failed_{timestamp}.xlsx'
    
    # Save the failed records to an Excel file with the generated file name
    failed_records.to_excel(f'{file_path}\\{report_file_name}', index=False)
else:
    print('Daily Validation Report Passed')  

## 2) Weekly Sanitation Data Validation between source and target ##

In [ ]:
#source data1
with database_1.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select div_str_id,task_desc, task_type_name, task_classification,sub_dpmt_name from
(
select div_str_id,a.task_desc,a.task_type, task_type_name, task_classification,sub_dpmt_name
from (SELECT * FROM santn_rotn_Task_mstr) a 
join (SELECT * FROM sub_dpmt_mpng) b using(sub_dpmt_id) 
join (SELECT * FROM str_dpmt_mpng) c on c.dpmt_name=b.prmry_dpmt_name 
left join (SELECT * FROM rolng_task_addtnl_info)  d on (a.santn_rotn_task_mstr_id=d.santn_rotn_task_mstr_id)  
where a.is_actv='Y' and c.is_actv='Y' 
and DATE(strt_date) >='2023-08-30' 
and DATE(strt_date) <='2023-09-05' 
and task_classification  like 'Weekly' and div_str_id ='53100515'
union all 
select div_str_id,a.task_desc,a.task_type, task_type_name, task_classification,sub_dpmt_name
from (SELECT * FROM santn_rotn_Task_mstr) a 
join (SELECT * FROM sub_dpmt_mpng) b using(sub_dpmt_id) 
join (SELECT * FROM str_dpmt_mpng) c on c.dpmt_name=b.prmry_dpmt_name 
left join (SELECT * FROM santn_rotn_task_schdl)  d on (a.santn_rotn_task_mstr_id=d.santn_rotn_task_mstr_id) 
where a.is_actv='Y' and c.is_actv='Y' 
and DATE(schd_strt_date) >='2023-08-30' 
and DATE(schd_strt_date) <='2023-09-05' 
and task_classification  like 'Weekly' and div_str_id ='53100515'
) 

 """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_ws_s1_df = pd.DataFrame(rows, columns = cols)    

In [ ]:
#Source data2
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select 
concat(division_number,store_number) AS div_str_id_2,
task_type as task_type_2,
task_classification as task_classification_2,
department_name as department_name_2,
task_description as task_description_2,
from task
where task_type='Sanitation' 
and task_classification like 'Weekly' 
and task_date='2023-09-04' 
and task_status='To Do'
        """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_ws_s2_df = pd.DataFrame(rows, columns = cols)    

In [ ]:
result_ws_s_df.head()

In [ ]:
# Joining the DataFrames using the specified conditions
result_ws_s_df = pd.merge(result_ws_s1_df, result_ws_s2_df, how='inner', left_on=[
    'div_str_id', 'task_type_name', 'task_classification','sub_dpmt_name', 'task_desc'
], right_on=[
    'div_str_id_2', 'task_type_2', 'task_classification_2', 'department_name_2', 'task_description_2'
])

# Select the desired columns in the result_df DataFrame
result_ws_s_df = result_ws_s_df[['div_str_id', 'task_desc', 'task_type_name', 'task_classification', 'sub_dpmt_name']]

# Rename the columns in the result_df DataFrame
result_ws_s_df = result_ws_s_df.rename(columns={
    'div_str_id': 'div_str_id_s',
    'task_type_name': 'task_type_name',
    'task_classification': 'task_classification_s',
    'task_desc': 'task_desc',
    'sub_dpmt_name': 'sub_dpmt_name'
})

In [ ]:
num_rows, num_columns = result_ws_s_df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")
result_ws_s_df.head()

In [ ]:
#Target data
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select 
concat(division_number,store_number) AS div_str_id,
task_type, task_classification, department_name, task_description
from task
where task_type<>'Green Rack' 
and task_classification  like 'Weekly'
and task_date='2023-09-05'
and task_type='Sanitation'
and concat(division_number, store_number) = '53100515'
        """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_ws_t_df = pd.DataFrame(rows, columns = cols)    

In [ ]:
num_rows, num_columns = result_ws_t_df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")
result_ws_t_df.head()

In [ ]:
# Merge the two DataFrames on 'div_str_id' and 'task_desc'
ws_merged_df = pd.merge(result_ws_s_df, result_ws_t_df, left_on=['div_str_id_s', 'task_desc'], right_on=['div_str_id', 'task_description'], how='outer')

# Create a new column 'ComparisonResult' based on your conditions
ws_merged_df['ComparisonResult'] = 'Mismatch'

# Condition 1: Matched
condition_matched = (
    (ws_merged_df['task_type_name'] == ws_merged_df['task_type']) &
    (ws_merged_df['task_classification_s'] == ws_merged_df['task_classification']) &
    #(ws_merged_df['sub_dpmt_name'] == ws_merged_df['department_name']) &
    (ws_merged_df['task_desc'] == ws_merged_df['task_description'])
    #(ws_merged_df['concatenated_s'] == ws_merged_df['concatenated_t'])
)
ws_merged_df.loc[condition_matched, 'ComparisonResult'] = 'Matched'

# Condition 2: Record NOT in Target
condition_not_in_target = (ws_merged_df['div_str_id_s'].notna()) & (ws_merged_df['div_str_id'].isna())
ws_merged_df.loc[condition_not_in_target, 'ComparisonResult'] = 'Record NOT in Target'

# Condition 3: Record NOT in Source
condition_not_in_source = (ws_merged_df['div_str_id'].notna()) & (ws_merged_df['div_str_id_s'].isna())
ws_merged_df.loc[condition_not_in_source, 'ComparisonResult'] = 'Record NOT in Source'

# Remove duplicates from ws_merged_df
ws_merged_df = ws_merged_df.drop_duplicates()

# Reset the index after removing duplicates (optional)
ws_merged_df.reset_index(drop=True, inplace=True)

# Filter records where 'ComparisonResult' is not 'Matched'
failed_records = ws_merged_df[ws_merged_df['ComparisonResult'] != 'Matched']

# Check if there are any failed records
if not failed_records.empty:
    print('Weekly Sanitation Validation Report Failed')

    # Get the current date as a timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    # Define the full path to save the report file
    file_path = r'C:\\Users\\AS81970\\Desktop\\2023 Q3\\Store Walk-in\\Reports'
    
    # Define the file name with the date appended
    report_file_name = f'Weekly_Sanitation_Validation_Report_Failed_{timestamp}.xlsx'
    
    # Save the failed records to an Excel file with the generated file name
    failed_records.to_excel(f'{file_path}\\{report_file_name}', index=False)
else:
    print('Weekly Sanitation Validation Report Passed')  

## 3) Weekly Rotation Data Validation between source and target ##

In [ ]:
#source data1
with database_1.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select div_str_id,task_desc, task_type_name, task_classification,sub_dpmt_name from
(
select div_str_id,a.task_desc,a.task_type, task_type_name, task_classification,sub_dpmt_name
from (SELECT * FROM santn_rotn_Task_mstr) a 
join (SELECT * FROM sub_dpmt_mpng) b using(sub_dpmt_id) 
join (SELECT * FROM str_dpmt_mpng) c on c.dpmt_name=b.prmry_dpmt_name 
left join (SELECT * FROM rolng_task_addtnl_info)  d on (a.santn_rotn_task_mstr_id=d.santn_rotn_task_mstr_id)  
where a.is_actv='Y' 
and c.is_actv='Y' 
and DATE(strt_date) >='2023-09-03' 
and DATE(strt_date) <='2023-09-05' 
and task_classification  like 'Weekly' 
and div_str_id ='53100515'
union all 
select div_str_id,a.task_desc,a.task_type, task_type_name, task_classification,sub_dpmt_name
from (SELECT * FROM santn_rotn_Task_mstr) a 
join (SELECT * FROM sub_dpmt_mpng) b using(sub_dpmt_id) 
join (SELECT * FROM str_dpmt_mpng) c on c.dpmt_name=b.prmry_dpmt_name 
left join (SELECT * FROM santn_rotn_task_schdl)  d on (a.santn_rotn_task_mstr_id=d.santn_rotn_task_mstr_id) 
where a.is_actv='Y' 
and c.is_actv='Y' 
and DATE(schd_strt_date) >='2023-09-03' 
and DATE(schd_strt_date) <='2023-09-05' 
and task_classification  like 'Weekly' 
and div_str_id ='53100515'
) 

 """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_wr_s1_df = pd.DataFrame(rows, columns = cols)    

In [ ]:
#Source data2
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select 
concat(division_number,store_number) AS div_str_id_2,
task_type as task_type_2,
task_classification as task_classification_2,
department_name as department_name_2,
task_description as task_description_2,
from task
where task_type='Rotation' 
and task_classification like 'Weekly' 
and task_date='2023-09-04' 
and task_status='To Do'
        """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_wr_s2_df = pd.DataFrame(rows, columns = cols)    

In [ ]:
# Joining the DataFrames using the specified conditions
result_wr_s_df = pd.merge(result_wr_s1_df, result_wr_s2_df, how='inner', left_on=[
    'div_str_id', 'task_type_name', 'task_classification','sub_dpmt_name', 'task_desc'
], right_on=[
    'div_str_id_2', 'task_type_2', 'task_classification_2', 'department_name_2', 'task_description_2'
])

# Select the desired columns in the result_df DataFrame
result_wr_s_df = result_wr_s_df[['div_str_id', 'task_desc', 'task_type_name', 'task_classification', 'sub_dpmt_name']]

# Rename the columns in the result_df DataFrame
result_wr_s_df = result_wr_s_df.rename(columns={
    'div_str_id': 'div_str_id_s',
    'task_type_name': 'task_type_name',
    'task_classification': 'task_classification_s',
    'task_desc': 'task_desc',
    'sub_dpmt_name': 'sub_dpmt_name'
})

In [ ]:
num_rows, num_columns = result_wr_s_df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")
result_wr_s_df.head()

In [ ]:
#Target data
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select 
concat(division_number,store_number) AS div_str_id,
task_type, task_classification, department_name, task_description
from task
where task_type<>'Green Rack' 
and task_classification  like 'Weekly'
and task_date='2023-09-05'
and task_type='Rotation'
and concat(division_number, store_number) = '53100515'
        """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_wr_t_df = pd.DataFrame(rows, columns = cols)    

In [ ]:
num_rows, num_columns = result_wr_t_df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")
result_wr_t_df.head()

In [ ]:
# Merge the two DataFrames on 'div_str_id' and 'task_desc'
wr_merged_df = pd.merge(result_wr_s_df, result_wr_t_df, left_on=['div_str_id_s', 'task_desc']
                        , right_on=['div_str_id', 'task_description'], how='outer')

# Create a new column 'ComparisonResult' based on your conditions
wr_merged_df['ComparisonResult'] = 'Mismatch'

# Condition 1: Matched
condition_matched = (
    (wr_merged_df['task_type_name'] == wr_merged_df['task_type']) &
    (wr_merged_df['task_classification_s'] == wr_merged_df['task_classification']) &
    #(wr_merged_df['sub_dpmt_name'] == wr_merged_df['department_name']) &
    (wr_merged_df['task_desc'] == wr_merged_df['task_description'])
    #(wr_merged_df['concatenated_s'] == wr_merged_df['concatenated_t'])
)
wr_merged_df.loc[condition_matched, 'ComparisonResult'] = 'Matched'

# Condition 2: Record NOT in Target
condition_not_in_target = (wr_merged_df['div_str_id_s'].notna()) & (wr_merged_df['div_str_id'].isna())
wr_merged_df.loc[condition_not_in_target, 'ComparisonResult'] = 'Record NOT in Target'

# Condition 3: Record NOT in Source
condition_not_in_source = (wr_merged_df['div_str_id'].notna()) & (wr_merged_df['div_str_id_s'].isna())
wr_merged_df.loc[condition_not_in_source, 'ComparisonResult'] = 'Record NOT in Source'

# Remove duplicates from wr_merged_df
wr_merged_df = wr_merged_df.drop_duplicates()

# Reset the index after removing duplicates (optional)
wr_merged_df.reset_index(drop=True, inplace=True)

# Filter records where 'ComparisonResult' is not 'Matched'
failed_records = wr_merged_df[wr_merged_df['ComparisonResult'] != 'Matched']

# Check if there are any failed records
if not failed_records.empty:
    print('Weekly Rotation Validation Report Failed')

    # Get the current date as a timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    # Define the full path to save the report file
    file_path = r'C:\\Users\\AS81970\\Desktop\\2023 Q3\\Store Walk-in\\Reports'
    
    # Define the file name with the date appended
    report_file_name = f'Weekly_Rotation_Validation_Report_Failed_{timestamp}.xlsx'
    
    # Save the failed records to an Excel file with the generated file name
    failed_records.to_excel(f'{file_path}\\{report_file_name}', index=False)
else:
    print('Weekly Rotation Validation Report Passed')  

## 4) 3xWeekly Data Validation between source and target ##

In [ ]:
#source data1
with database_1.snapshot() as snapshot:
    result = snapshot.execute_sql("""
        SELECT *
        FROM (
            SELECT div_str_id, a.task_desc, a.task_type, task_type_name, task_classification, sub_dpmt_name
            FROM santn_rotn_Task_mstr a
            JOIN sub_dpmt_mpng b USING (sub_dpmt_id)
            JOIN str_dpmt_mpng c ON c.dpmt_name = b.prmry_dpmt_name
            LEFT JOIN rolng_task_addtnl_info d ON (a.santn_rotn_task_mstr_id = d.santn_rotn_task_mstr_id)
            WHERE a.is_actv = 'Y' 
              AND c.is_actv = 'Y' 
              AND DATE(strt_date) = '2023-08-11' --and DATE(strt_date) <= '2023-08-11' 
              AND div_str_id = '53100515' 
              AND task_classification LIKE '3x Weekly'
            UNION ALL
            SELECT div_str_id, a.task_desc, a.task_type, task_type_name, task_classification, sub_dpmt_name
            FROM santn_rotn_Task_mstr a
            JOIN sub_dpmt_mpng b USING (sub_dpmt_id)
            JOIN str_dpmt_mpng c ON c.dpmt_name = b.prmry_dpmt_name
            LEFT JOIN santn_rotn_task_schdl d ON (a.santn_rotn_task_mstr_id = d.santn_rotn_task_mstr_id)
            WHERE a.is_actv = 'Y' 
              AND c.is_actv = 'Y' 
              AND DATE(schd_strt_date) >= '2023-08-11' and DATE(schd_strt_date) <= '2023-08-11' 
              AND div_str_id = '53100515' 
              AND task_classification LIKE '3x Weekly'
        ) 
    """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_3w_s1_df = pd.DataFrame(rows, columns = cols)    

In [ ]:
#Source data2
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select distinct concat(division_number,store_number) AS div_str_id,
from task where task_date='2023-08-11'
        """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_3w_s2_df = pd.DataFrame(rows, columns = cols)    

In [ ]:
# Joining the DataFrames using the specified conditions
result_3w_s_df = pd.merge(result_3w_s1_df, result_3w_s2_df,left_on=['div_str_id'], right_on=['div_str_id'], how='inner')

# Select the desired columns in the result_df DataFrame
result_3w_s_df = result_3w_s_df[['div_str_id', 'task_desc', 'task_type_name', 'task_classification', 'sub_dpmt_name']]

# Rename the columns in the result_df DataFrame
result_3w_s_df = result_3w_s_df.rename(columns={
    'div_str_id': 'div_str_id_s',
    'task_type_name': 'task_type_name',
    'task_classification': 'task_classification_s',
    'task_desc': 'task_desc',
    'sub_dpmt_name': 'sub_dpmt_name'
})

In [ ]:
result_3w_s_df.head()

In [ ]:
#Target data
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select 
concat(division_number,store_number) AS div_str_id,
task_type, task_classification, department_name, task_description
from task
where task_type<>'Green Rack' 
and task_classification  like '3x Weekly'
and task_date='2023-08-11'
and concat(division_number, store_number) = '53100515'
        """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_3w_t_df = pd.DataFrame(rows, columns = cols)    

In [ ]:
result_3w_t_df.head()

In [ ]:
# Merge the two DataFrames on 'div_str_id' and 'task_desc'
w3_merged_df = pd.merge(result_3w_s_df, result_3w_t_df, left_on=['div_str_id_s', 'task_desc']
                        , right_on=['div_str_id', 'task_description'], how='outer')

# Create a new column 'ComparisonResult' based on your conditions
w3_merged_df['ComparisonResult'] = 'Mismatch'

# Condition 1: Matched
condition_matched = (
    (w3_merged_df['task_type_name'] == w3_merged_df['task_type']) &
    (w3_merged_df['task_classification_s'] == w3_merged_df['task_classification']) &
    #(w3_merged_df['sub_dpmt_name'] == w3_merged_df['department_name']) &
    (w3_merged_df['task_desc'] == w3_merged_df['task_description'])
    #(w3_merged_df['concatenated_s'] == w3_merged_df['concatenated_t'])
)
w3_merged_df.loc[condition_matched, 'ComparisonResult'] = 'Matched'

# Condition 2: Record NOT in Target
condition_not_in_target = (w3_merged_df['div_str_id_s'].notna()) & (w3_merged_df['div_str_id'].isna())
w3_merged_df.loc[condition_not_in_target, 'ComparisonResult'] = 'Record NOT in Target'

# Condition 3: Record NOT in Source
condition_not_in_source = (w3_merged_df['div_str_id'].notna()) & (w3_merged_df['div_str_id_s'].isna())
w3_merged_df.loc[condition_not_in_source, 'ComparisonResult'] = 'Record NOT in Source'

# Remove duplicates from w3_merged_df
w3_merged_df = w3_merged_df.drop_duplicates()

# Reset the index after removing duplicates (optional)
w3_merged_df.reset_index(drop=True, inplace=True)

# Filter records where 'ComparisonResult' is not 'Matched'
failed_records = w3_merged_df[w3_merged_df['ComparisonResult'] != 'Matched']

# Check if there are any failed records
if not failed_records.empty:
    print('3xWeekly Validation Report Failed')

    # Get the current date as a timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    # Define the full path to save the report file
    file_path = r'C:\\Users\\AS81970\\Desktop\\2023 Q3\\Store Walk-in\\Reports'
    
    # Define the file name with the date appended
    report_file_name = f'3xWeekly_Validation_Report_Failed_{timestamp}.xlsx'
    
    # Save the failed records to an Excel file with the generated file name
    failed_records.to_excel(f'{file_path}\\{report_file_name}', index=False)
else:
    print('3xWeekly Validation Report Passed')  

## 5) Periodic Data Validation between source and target ##

In [ ]:
#source data1
with database_1.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select div_str_id,task_desc, task_type_name, task_classification,sub_dpmt_name from
(
select div_str_id,a.task_desc,a.task_type, task_type_name, task_classification,sub_dpmt_name
from (SELECT * FROM santn_rotn_Task_mstr) a 
join (SELECT * FROM sub_dpmt_mpng) b using(sub_dpmt_id) 
join (SELECT * FROM str_dpmt_mpng) c on c.dpmt_name=b.prmry_dpmt_name 
left join (SELECT * FROM rolng_task_addtnl_info)  d on (a.santn_rotn_task_mstr_id=d.santn_rotn_task_mstr_id)  
where a.is_actv='Y' and c.is_actv='Y' 
and DATE(strt_date) >='2023-08-30' 
and DATE(strt_date) <='2023-09-05' 
and task_classification  like 'Periodic' and div_str_id ='53100515'
union all 
select div_str_id,a.task_desc,a.task_type, task_type_name, task_classification,sub_dpmt_name
from (SELECT * FROM santn_rotn_Task_mstr) a 
join (SELECT * FROM sub_dpmt_mpng) b using(sub_dpmt_id) 
join (SELECT * FROM str_dpmt_mpng) c on c.dpmt_name=b.prmry_dpmt_name 
left join (SELECT * FROM santn_rotn_task_schdl)  d on (a.santn_rotn_task_mstr_id=d.santn_rotn_task_mstr_id) 
where a.is_actv='Y' and c.is_actv='Y' 
and DATE(schd_strt_date) >='2023-08-30' 
and DATE(schd_strt_date) <='2023-09-05' 
and task_classification  like 'Periodic' and div_str_id ='53100515'
) 

 """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_p_s1_df = pd.DataFrame(rows, columns = cols)   

In [ ]:
#Source data2
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select 
concat(division_number,store_number) AS div_str_id_2,
task_type as task_type_2,
task_classification as task_classification_2,
department_name as department_name_2,
task_description as task_description_2,
from task
where task_type='Sanitation' 
and task_classification like 'Periodic' 
and task_date='2023-09-04' 
and task_status='To Do'
        """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_p_s2_df = pd.DataFrame(rows, columns = cols)    

In [ ]:
# Joining the DataFrames using the specified conditions
result_p_s_df = pd.merge(result_p_s1_df, result_p_s2_df, how='inner', left_on=[
    'div_str_id', 'task_type_name', 'task_classification','sub_dpmt_name', 'task_desc'
], right_on=[
    'div_str_id_2', 'task_type_2', 'task_classification_2', 'department_name_2', 'task_description_2'
])

# Select the desired columns in the result_df DataFrame
result_p_s_df = result_p_s_df[['div_str_id', 'task_desc', 'task_type_name', 'task_classification', 'sub_dpmt_name']]

# Rename the columns in the result_df DataFrame
result_p_s_df = result_p_s_df.rename(columns={
    'div_str_id': 'div_str_id_s',
    'task_type_name': 'task_type_name',
    'task_classification': 'task_classification_s',
    'task_desc': 'task_desc',
    'sub_dpmt_name': 'sub_dpmt_name'
})

In [ ]:
num_rows, num_columns = result_p_s_df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")
result_p_s_df.head()

In [ ]:
#Target data
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select 
concat(division_number,store_number) AS div_str_id,
task_type, task_classification, department_name, task_description
from task
where task_type<>'Green Rack' 
and task_classification  like 'Periodic'
and task_date='2023-09-05'
and concat(division_number, store_number) = '53100515'
        """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_p_t_df = pd.DataFrame(rows, columns = cols)    

In [ ]:
num_rows, num_columns = result_p_t_df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")
result_p_t_df.head()

In [ ]:
# Merge the two DataFrames on 'div_str_id' and 'task_desc'
p_merged_df = pd.merge(result_p_s_df, result_p_t_df, left_on=['div_str_id_s', 'task_desc'], right_on=['div_str_id', 'task_description'], how='outer')

# Create a new column 'ComparisonResult' based on your conditions
p_merged_df['ComparisonResult'] = 'Mismatch'

# Condition 1: Matched
condition_matched = (
    (p_merged_df['task_type_name'] == p_merged_df['task_type']) &
    (p_merged_df['task_classification_s'] == p_merged_df['task_classification']) &
    #(p_merged_df['sub_dpmt_name'] == p_merged_df['department_name']) &
    (p_merged_df['task_desc'] == p_merged_df['task_description'])
    #(p_merged_df['concatenated_s'] == p_merged_df['concatenated_t'])
)
p_merged_df.loc[condition_matched, 'ComparisonResult'] = 'Matched'

# Condition 2: Record NOT in Target
condition_not_in_target = (p_merged_df['div_str_id_s'].notna()) & (p_merged_df['div_str_id'].isna())
p_merged_df.loc[condition_not_in_target, 'ComparisonResult'] = 'Record NOT in Target'

# Condition 3: Record NOT in Source
condition_not_in_source = (p_merged_df['div_str_id'].notna()) & (p_merged_df['div_str_id_s'].isna())
p_merged_df.loc[condition_not_in_source, 'ComparisonResult'] = 'Record NOT in Source'

# Remove duplicates from p_merged_df
p_merged_df = p_merged_df.drop_duplicates()

# Reset the index after removing duplicates (optional)
p_merged_df.reset_index(drop=True, inplace=True)

# Filter records where 'ComparisonResult' is not 'Matched'
failed_records = p_merged_df[p_merged_df['ComparisonResult'] != 'Matched']

# Check if there are any failed records
if not failed_records.empty:
    print('Periodic Validation Report Failed')

    # Get the current date as a timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    # Define the full path to save the report file
    file_path = r'C:\\Users\\AS81970\\Desktop\\2023 Q3\\Store Walk-in\\Reports'
    
    # Define the file name with the date appended
    report_file_name = f'Periodic_Validation_Report_Failed_{timestamp}.xlsx'
    
    # Save the failed records to an Excel file with the generated file name
    failed_records.to_excel(f'{file_path}\\{report_file_name}', index=False)
else:
    print('Periodic Validation Report Passed')  

## 6) Quaterly Data Validation between source and target ##

In [ ]:
#database_id_1 = "udm-db"  & instance_id = "nc-event-sink-instance" (Source Query)

#Source Data
with database_1.snapshot() as snapshot:
    result = snapshot.execute_sql("""
        SELECT * FROM (
            SELECT
                div_str_id AS div_str_id_s,
                task_type_name,
                task_classification AS task_classification_s,
                a.task_desc,
            FROM
                (SELECT * FROM santn_rotn_task_mstr) a
                JOIN (SELECT * FROM sub_dpmt_mpng) b USING (sub_dpmt_id)
                JOIN (SELECT * FROM str_dpmt_mpng) c ON c.dpmt_name = b.prmry_dpmt_name
                LEFT JOIN (SELECT * FROM rolng_task_addtnl_info) d
                    ON (a.santn_rotn_task_mstr_id = d.santn_rotn_task_mstr_id)
            WHERE
                a.is_actv = 'Y'
                AND c.is_actv = 'Y'
                and DATE(strt_date) >='2023-08-11' 
                and DATE(strt_date) <='2023-08-11' 
                AND task_classification LIKE 'Quarterly'
                AND div_str_id = '53100515'
            UNION ALL
            SELECT
                div_str_id AS div_str_id_s,
                task_type_name,
                task_classification AS task_classification_s,
                a.task_desc,
            FROM
                (SELECT * FROM santn_rotn_task_mstr) a
                JOIN (SELECT * FROM sub_dpmt_mpng) b USING (sub_dpmt_id)
                JOIN (SELECT * FROM str_dpmt_mpng) c ON c.dpmt_name = b.prmry_dpmt_name
                LEFT JOIN (SELECT * FROM santn_rotn_task_schdl) d
                    ON (a.santn_rotn_task_mstr_id = d.santn_rotn_task_mstr_id)
            WHERE
                a.is_actv = 'Y'
                AND c.is_actv = 'Y'
                and DATE(schd_strt_date) >='2023-08-11' 
                and DATE(schd_strt_date) <='2023-08-11'
                AND task_classification LIKE 'Quarterly'
                AND div_str_id = '53100515'
        ) Daily_S
    """)
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_q_s_df = pd.DataFrame(rows, columns = cols)

In [ ]:
num_rows, num_columns = result_q_s_df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")
result_q_s_df.head()

In [ ]:
#database_id_2 = "storewalks-db"  & instance_id = "nc-event-sink-instance"
#Target Data
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
        SELECT
            CONCAT(division_number, store_number) AS div_str_id,
            task_type,
            task_classification,
            task_description,
        FROM
            task
        WHERE
            task_type <> 'Green Rack'
            AND task_classification LIKE 'Quarterly'
            AND task_date = '2023-08-11'
            AND CONCAT(division_number, store_number) = '53100515'
    """)
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_q_t_df = pd.DataFrame(rows, columns=cols)


In [ ]:
num_rows, num_columns = result_q_t_df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")
result_q_t_df.head()

In [ ]:
# Merge the two DataFrames on 'div_str_id' and 'task_desc'
q_merged_df = pd.merge(result_q_s_df, result_q_t_df, left_on=['div_str_id_s', 'task_desc'], right_on=['div_str_id', 'task_description'], how='outer')

# Create a new column 'ComparisonResult' based on your conditions
q_merged_df['ComparisonResult'] = 'Mismatch'

# Condition 1: Matched
condition_matched = (
    (q_merged_df['task_type_name'] == q_merged_df['task_type']) &
    (q_merged_df['task_classification_s'] == q_merged_df['task_classification']) &
    #(q_merged_df['sub_dpmt_name'] == q_merged_df['department_name']) &
    (q_merged_df['task_desc'] == q_merged_df['task_description'])
    #(q_merged_df['concatenated_s'] == q_merged_df['concatenated_t'])
)
q_merged_df.loc[condition_matched, 'ComparisonResult'] = 'Matched'

# Condition 2: Record NOT in Target
condition_not_in_target = (q_merged_df['div_str_id_s'].notna()) & (q_merged_df['div_str_id'].isna())
q_merged_df.loc[condition_not_in_target, 'ComparisonResult'] = 'Record NOT in Target'

# Condition 3: Record NOT in Source
condition_not_in_source = (q_merged_df['div_str_id'].notna()) & (q_merged_df['div_str_id_s'].isna())
q_merged_df.loc[condition_not_in_source, 'ComparisonResult'] = 'Record NOT in Source'

# Remove duplicates from q_merged_df
q_merged_df = q_merged_df.drop_duplicates()

# Reset the index after removing duplicates (optional)
q_merged_df.reset_index(drop=True, inplace=True)

# Filter records where 'ComparisonResult' is not 'Matched'
failed_records = q_merged_df[q_merged_df['ComparisonResult'] != 'Matched']

# Check if there are any failed records
if not failed_records.empty:
    print('Quaterly Validation Report Failed')

    # Get the current date as a timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    # Define the full path to save the report file
    file_path = r'C:\\Users\\AS81970\\Desktop\\2023 Q3\\Store Walk-in\\Reports'
    
    # Define the file name with the date appended
    report_file_name = f'Quaterly_Validation_Report_Failed_{timestamp}.xlsx'
    
    # Save the failed records to an Excel file with the generated file name
    failed_records.to_excel(f'{file_path}\\{report_file_name}', index=False)
else:
    print('Quaterly Validation Report Passed')  

###  -------------- Counts  -------------- ###

## 1) Daily Counts Validation between source and target ##

In [4]:
#Source Counts
with database_1.snapshot() as snapshot:
    result = snapshot.execute_sql("""
 select sum(count), 'daily_s'  
from    
 (
 (select  distinct count(*) as count,div_str_id, task_type_name, task_classification,sub_dpmt_name 
            FROM
                (SELECT * FROM santn_rotn_task_mstr) a
                JOIN (SELECT * FROM sub_dpmt_mpng) b USING (sub_dpmt_id)
                JOIN (SELECT * FROM str_dpmt_mpng) c ON c.dpmt_name = b.prmry_dpmt_name
                LEFT JOIN (SELECT * FROM rolng_task_addtnl_info) d
                    ON (a.santn_rotn_task_mstr_id = d.santn_rotn_task_mstr_id)
            WHERE
                a.is_actv = 'Y'
                AND c.is_actv = 'Y'
                AND DATE(strt_date) = '2023-09-05'
                AND DATE(strt_date) <= '2023-09-05'
                AND task_classification LIKE 'Daily'
group by div_str_id, task_type_name, task_classification,sub_dpmt_name)  
union all
(select  distinct count(*) as count,div_str_id, task_type_name, task_classification,sub_dpmt_name 
            FROM
                (SELECT * FROM santn_rotn_task_mstr) a
                JOIN (SELECT * FROM sub_dpmt_mpng) b USING (sub_dpmt_id)
                JOIN (SELECT * FROM str_dpmt_mpng) c ON c.dpmt_name = b.prmry_dpmt_name
                LEFT JOIN (SELECT * FROM santn_rotn_task_schdl) d
                    ON (a.santn_rotn_task_mstr_id = d.santn_rotn_task_mstr_id)
            WHERE
                a.is_actv = 'Y'
                AND c.is_actv = 'Y'
                AND DATE(schd_strt_date) = '2023-09-05'
                AND DATE(schd_strt_date) <= '2023-09-05'
                AND task_classification LIKE 'Daily'
group by div_str_id, task_type_name, task_classification,sub_dpmt_name) 
)   
    
        """)
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_s_c_df = pd.DataFrame(rows, columns = cols)

In [5]:
#Target Counts
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select sum(count1), 'daily_t' 
from 
(select count(*) as count1,division_number, store_number, task_type, task_classification,department_name, task_date, task_status 
from  task
where task_type<>'Green Rack' and task_classification like 'Daily' and task_date='2023-09-05'
group by division_number, store_number, task_type, task_classification,department_name, task_date, task_status
)
        """)
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_t_c_df = pd.DataFrame(rows, columns = cols)

## 2) Weekly Sanitation Counts Validation between source and target ##

In [6]:
#source data1
with database_1.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select div_str_id,task_desc, task_type_name, task_classification,sub_dpmt_name from
(
select div_str_id,a.task_desc,a.task_type, task_type_name, task_classification,sub_dpmt_name
from (SELECT * FROM santn_rotn_Task_mstr) a 
join (SELECT * FROM sub_dpmt_mpng) b using(sub_dpmt_id) 
join (SELECT * FROM str_dpmt_mpng) c on c.dpmt_name=b.prmry_dpmt_name 
left join (SELECT * FROM rolng_task_addtnl_info)  d on (a.santn_rotn_task_mstr_id=d.santn_rotn_task_mstr_id)  
where a.is_actv='Y' and c.is_actv='Y' 
and DATE(strt_date) >='2023-08-30' 
and DATE(strt_date) <='2023-09-05' 
and task_classification  like 'Weekly' --and div_str_id ='53100515'
union all 
select div_str_id,a.task_desc,a.task_type, task_type_name, task_classification,sub_dpmt_name
from (SELECT * FROM santn_rotn_Task_mstr) a 
join (SELECT * FROM sub_dpmt_mpng) b using(sub_dpmt_id) 
join (SELECT * FROM str_dpmt_mpng) c on c.dpmt_name=b.prmry_dpmt_name 
left join (SELECT * FROM santn_rotn_task_schdl)  d on (a.santn_rotn_task_mstr_id=d.santn_rotn_task_mstr_id) 
where a.is_actv='Y' and c.is_actv='Y' 
and DATE(schd_strt_date) >='2023-08-30' 
and DATE(schd_strt_date) <='2023-09-05' 
and task_classification  like 'Weekly' --and div_str_id ='53100515'
) 

 """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_ws_s1_c_df = pd.DataFrame(rows, columns = cols)    

In [7]:
#Source data2
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select 
concat(division_number,store_number) AS div_str_id_2,
task_type as task_type_2,
task_classification as task_classification_2,
department_name as department_name_2,
task_description as task_description_2,
from task
where task_type='Sanitation' 
and task_classification like 'Weekly' 
and task_date='2023-09-04' 
and task_status='To Do'
        """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_ws_s2_c_df = pd.DataFrame(rows, columns = cols)    

In [8]:
# Joining the DataFrames using the specified conditions
result_ws_c_df = pd.merge(result_ws_s1_c_df, result_ws_s2_c_df, how='inner', left_on=[
    'div_str_id', 'task_type_name', 'task_classification','sub_dpmt_name', 'task_desc'
], right_on=[
    'div_str_id_2', 'task_type_2', 'task_classification_2', 'department_name_2', 'task_description_2'
])

# Select the desired columns in the result_df DataFrame
result_ws_c_df = result_ws_c_df[['div_str_id', 'task_desc', 'task_type_name', 'task_classification', 'sub_dpmt_name']]

# Rename the columns in the result_df DataFrame
result_ws_c_df = result_ws_c_df.rename(columns={
    'div_str_id': 'div_str_id_s',
    'task_type_name': 'task_type_name',
    'task_classification': 'task_classification_s',
    'task_desc': 'task_desc',
    'sub_dpmt_name': 'sub_dpmt_name'
})

# Group by 'div_str_id_s' and count the occurrences
result_ws_c_count_df = result_ws_c_df.groupby('div_str_id_s').size().reset_index(name='count')

# Calculate the sum of 'count'
total_count = result_ws_c_count_df['count'].sum()

# Create a DataFrame with 'count' and 'weekly_s' columns
result_label_df = pd.DataFrame({'count': [total_count], 'label': ['weekly_s']})

#result_label_df.head()

In [9]:
#Target data
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select sum(count1), 'weekly_t' 
from 
(select count(*) as count1, division_number,store_number,task_type, task_classification, department_name, task_description
from task
where task_type<>'Green Rack' 
and task_classification  like 'Weekly'
and task_date='2023-09-05'
and task_type='Sanitation'
group by division_number, store_number, task_type, task_classification,department_name, task_date, task_status,task_description
)

    """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_ws_t_c_df = pd.DataFrame(rows, columns = cols)    

## 2) Weekly Rotation Counts Validation between source and target ##

In [17]:
#source data1
with database_1.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select div_str_id,task_desc, task_type_name, task_classification,sub_dpmt_name from
(
select div_str_id,a.task_desc,a.task_type, task_type_name, task_classification,sub_dpmt_name
from (SELECT * FROM santn_rotn_Task_mstr) a 
join (SELECT * FROM sub_dpmt_mpng) b using(sub_dpmt_id) 
join (SELECT * FROM str_dpmt_mpng) c on c.dpmt_name=b.prmry_dpmt_name 
left join (SELECT * FROM rolng_task_addtnl_info)  d on (a.santn_rotn_task_mstr_id=d.santn_rotn_task_mstr_id)  
where a.is_actv='Y' and c.is_actv='Y' 
and DATE(strt_date) >='2023-09-03' 
and DATE(strt_date) <='2023-09-05' 
and task_classification  like 'Weekly' --and div_str_id ='53100515'
union all 
select div_str_id,a.task_desc,a.task_type, task_type_name, task_classification,sub_dpmt_name
from (SELECT * FROM santn_rotn_Task_mstr) a 
join (SELECT * FROM sub_dpmt_mpng) b using(sub_dpmt_id) 
join (SELECT * FROM str_dpmt_mpng) c on c.dpmt_name=b.prmry_dpmt_name 
left join (SELECT * FROM santn_rotn_task_schdl)  d on (a.santn_rotn_task_mstr_id=d.santn_rotn_task_mstr_id) 
where a.is_actv='Y' and c.is_actv='Y' 
and DATE(schd_strt_date) >='2023-09-03' 
and DATE(schd_strt_date) <='2023-09-05' 
and task_classification  like 'Weekly' --and div_str_id ='53100515'
) 

 """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_wr_s1_c_df = pd.DataFrame(rows, columns = cols)    

In [18]:
#Source data2
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select 
concat(division_number,store_number) AS div_str_id_2,
task_type as task_type_2,
task_classification as task_classification_2,
department_name as department_name_2,
task_description as task_description_2,
from task
where task_type='Rotation' 
and task_classification like 'Weekly' 
and task_date='2023-09-04' 
and task_status='To Do'
        """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_wr_s2_c_df = pd.DataFrame(rows, columns = cols)    

In [27]:
# Joining the DataFrames using the specified conditions
result_wr_c_df = pd.merge(result_wr_s1_c_df, result_wr_s2_c_df, how='inner', left_on=[
    'div_str_id', 'task_type_name', 'task_classification','sub_dpmt_name', 'task_desc'
], right_on=[
    'div_str_id_2', 'task_type_2', 'task_classification_2', 'department_name_2', 'task_description_2'
])

# Select the desired columns in the result_df DataFrame
result_wr_c_df = result_wr_c_df[['div_str_id', 'task_desc', 'task_type_name', 'task_classification', 'sub_dpmt_name']]

# Rename the columns in the result_df DataFrame
result_wr_c_df = result_wr_c_df.rename(columns={
    'div_str_id': 'div_str_id_s',
    'task_type_name': 'task_type_name',
    'task_classification': 'task_classification_s',
    'task_desc': 'task_desc',
    'sub_dpmt_name': 'sub_dpmt_name'
})

# Group by 'div_str_id_s' and count the occurrences
result_wr_c_count_df = result_wr_c_df.groupby('div_str_id_s').size().reset_index(name='count')

# Calculate the sum of 'count'
total_count2 = result_wr_c_count_df['count'].sum()

# Create a DataFrame with 'count' and 'weekly_s' columns
result_label2_df = pd.DataFrame({'count': [total_count2], 'label': ['weekly_r_s']})

result_label2_df.head()

,count,label
0,0,weekly_r_s


In [25]:
#Target data
with database_2.snapshot() as snapshot:
    result = snapshot.execute_sql("""
select sum(count1), 'weekly_r_t' 
from 
(select count(*) as count1, division_number,store_number,task_type, task_classification, department_name, task_description
from task
where task_type<>'Green Rack' 
and task_classification  like 'Weekly'
and task_date='2023-09-05'
and task_type='Rotation'
group by division_number, store_number, task_type, task_classification,department_name, task_date, task_status,task_description
)

    """)
    
    
    # Stream in rows
    rows = list()
    for row in result:
        rows.append(row)

    # Get column names
    cols = [x.name for x in result.fields]

    # Convert to pandas dataframe
    result_wr_t_c_df = pd.DataFrame(rows, columns = cols)    

In [26]:
result_wr_t_c_df.head()

,,
0,4972,weekly_r_t


## Writing All counts to Report ##

In [28]:
# Get the current timestamp
timestamp_c = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Define the file path
file_path_c = f"C:\\Users\\AS81970\\Desktop\\2023 Q3\\Store Walk-in\\Reports\\Count_{timestamp_c}.csv"

# Write the DataFrames to the CSV file
result_s_c_df.to_csv(file_path_c, index=False) #daily
result_t_c_df.to_csv(file_path_c, mode='a', header=False, index=False)  #daily
result_label_df.to_csv(file_path_c, mode='a', header=False, index=False) #weekly S
result_ws_t_c_df.to_csv(file_path_c, mode='a', header=False, index=False) #weekly S
result_label2_df.to_csv(file_path_c, mode='a', header=False, index=False) #weekly R
result_wr_t_c_df.to_csv(file_path_c, mode='a', header=False, index=False) #weekly R


In [ ]:
Counts_Report_Date 
 Daily count Source - Done
 Daily count Target - Done
 Weekly Sanitation count Source - Done
 Weekly Sanitation count Target -Done
 Weekly Rotation count Source - Done 
 Weekly Rotation count Target - Done
 3xWeekly count Source - 
 3xWeekly count Target - 
 Periodic count Source - 
 Periodic count Target - 
 
 ------------------------
Daily_Validation_Report_Date - Done
Weekly_Sanitation_Validation_Report_Date - Done
Weekly_Rotation_Validation_Report_Date - Done
3xWeekly_ReportValidation_Report_Date - DOne
Periodic_Validation_Report_Date - Done
Quaterly_Validation_Report_Date - Done

 

In [ ]:
#merged_df.to_csv("C:\\Users\\AS81970\\Desktop\\tm.csv", index=False)